# Unidad 04
## Ejercicios Podio

In [ ]:
#r "nuget: Microsoft.DotNet.Interactive.SqlServer, *-*"

In [ ]:
foreach (var line in System.IO.File.ReadLines(".env"))
{
    var parts = line.Split('=');
    if (parts.Length == 2)
    {
        var key = parts[0].Trim();
        var value = parts[1].Trim();
        Environment.SetEnvironmentVariable(key, value);
    }
}

string connectionString = "Server=" + Environment.GetEnvironmentVariable("Server") + ";Database=" + Environment.GetEnvironmentVariable("Database") + ";User Id=" + Environment.GetEnvironmentVariable("User") + ";Password=" + Environment.GetEnvironmentVariable("Password") + ";Encrypt=True;TrustServerCertificate=True;Connection Timeout=30;";


In [ ]:
#!connect mssql --kernel-name Pampero @csharp:connectionString

In [ ]:
DROP TABLE IF EXISTS dbo.podio;

CREATE TABLE dbo.podio ( 
nombre nvarchar(50) PRIMARY KEY, 
pais nvarchar(50), 
tiempo numeric(18,2) 
); 

In [ ]:
INSERT INTO dbo.podio (nombre, pais, tiempo)
VALUES 
('Aleksandr Sadovnikov', 'RUS', 51.84),
('Chad Le Clos', 'RSA', 51.14),
('Joseph Schooling', 'SGP', 50.39),
('Laszlo Cseh', 'HUN', 51.14),
('Medhy Metella', 'FRA', 51.58),
('Michael Phelps', 'USA', 51.14),
('Tom Shields', 'USA', 51.73),
('Zhuhao Li', 'CHN', 51.26);


1) Obtenga el podio utilizando TOP ( WITH TIES ).

In [ ]:
USE Pampero;
SELECT TOP 3 WITH TIES nombre, pais, tiempo
FROM dbo.podio
ORDER BY tiempo

2) Suponga que no puede utilizar TOP

In [ ]:
SELECT nombre, pais, tiempo
FROM (SELECT nombre, pais, tiempo,
    ROW_NUMBER() OVER(ORDER BY tiempo) AS rownum,
    RANK() OVER(ORDER BY tiempo) AS rnk,
    DENSE_RANK() OVER(ORDER BY tiempo) AS densernk,
    NTILE(100) OVER(ORDER BY tiempo) AS ntile100
FROM dbo.podio) AS P
WHERE rnk <= 3

3) Agregue al resultado una columna extra que indique la medalla recibida por cada participante (el primero recibe oro, el segundo plata y el tercero bronce, el resto no recibe medalla). Resultado esperado:

|nombre              |pais|tiempo|Medalla|
|--------------------|----|------|-------|
|Joseph Schooling    |SGP |50.39 |Oro    |
|Laszlo Cseh         |HUN |51.14 |Plata  |
|Chad Le Clos        |RSA |51.14 |Plata  |
|Michael Phelps      |USA |51.14 |Plata  |
|Zhuhao Li           |CHN |51.26 |null   |
|Medhy Metella       |FRA |51.58 |null   |
|Tom Shields         |USA |51.73 |null   |
|Aleksandr Sadovnikov|RUS |51.84 |null   |



In [ ]:
SELECT nombre, pais, tiempo, 
CASE RANK() OVER(ORDER BY tiempo) 
WHEN 1 THEN 'Oro'
WHEN 2 THEN 'Plata'
WHEN 3 THEN 'Bronce'
END AS Medalla
FROM podio
ORDER BY tiempo

4) Calcule las diferencias entre los corredores por orden de llegada. Resultado esperado:

|nombre              |tiempo|Dif |
|--------------------|------|----|
|Joseph Schooling    |50.39 |null|
|Chad Le Clos        |51.14 |0.75|
|Laszlo Cseh         |51.14 |0.00|
|Michael Phelps      |51.14 |0.00|
|Zhuhao Li           |51.26 |0.12|
|Medhy Metella       |51.58 |0.32|
|Tom Shields         |51.73 |0.15|
|Aleksandr Sadovnikov|51.84 |0.11|



In [ ]:
SELECT nombre, tiempo,
	tiempo - LAG(tiempo) 
OVER(ORDER BY tiempo, nombre) AS Dif
FROM dbo.podio;

5) Calcule las diferencias entre los corredores con respecto al ganador. Resultado esperado:

|nombre              |tiempo|Dif |
|--------------------|------|----|
|Joseph Schooling    |50.39 |0.00|
|Chad Le Clos        |51.14 |0.75|
|Laszlo Cseh         |51.14 |0.75|
|Michael Phelps      |51.14 |0.75|
|Zhuhao Li           |51.26 |0.87|
|Medhy Metella       |51.58 |1.19|
|Tom Shields         |51.73 |1.34|
|Aleksandr Sadovnikov|51.84 |1.45|



In [ ]:
SELECT nombre, tiempo,
	tiempo - FIRST_VALUE(tiempo) OVER(ORDER BY tiempo, nombre) AS Dif
FROM dbo.podio;

6) ¿Qué corredores tienen tiempo por encima del promedio de su país?

In [ ]:
SELECT nombre
FROM (SELECT nombre, pais, tiempo,
AVG(tiempo) OVER(PARTITION BY pais) as promediopais
FROM dbo.podio) AS P
WHERE tiempo > promediopais